# Grand canonical ensemble transition-matrix Monte Carlo with SPC/E

This example is similar to the Lennard Jones tutorial, except this time we simulate the SPC/E water model.

In [1]:
import sys
import unittest
import feasst
sys.path.insert(0, feasst.install_dir() + '/plugin/flat_histogram/tutorial/')
import fh
sys.path.insert(0, feasst.install_dir() + '/plugin/ewald/tutorial/')
import spce

class TestFlatHistogramLJ(unittest.TestCase):
    """Test flat histogram grand canonical ensemble Monte Carlo simulations"""
    def test_serial_5max(self):
        """Compare the free energies and potential energies with the NIST SRSW
        https://www.nist.gov/programs-projects/nist-standard-reference-simulation-website
        https://mmlapps.nist.gov/srs/LJ_PURE/eostmmc.htm
        """
        config = feasst.Configuration(
            feasst.MakeDomain(feasst.args({"cubic_box_length": "20"})),
            feasst.args({"particle_type": feasst.install_dir() +  "/forcefield/data.spce",
                         "physical_constants": "CODATA2010"}))
        R = config.model_params().physical_constants().ideal_gas_constant()
        criteria = fh.criteria_flathist(temperature=525*R/1000.,
                                        chemical_potential=-35.294567543492,
                                        macro_min=0, macro_max=5, tmmc=True)
        monte_carlo = fh.monte_carlo(criteria=criteria,
                                     system=spce.system(config))
        
        lnpi_previous = [
            [-2.8646724680467574586, 0.013169999365463097227],
            [-1.9398627751910603179, 0.01178939015260997529],
            [-1.5807236756003240075, 0.016351895458997890015],
            [-1.5052807379855992487, 0.0066184482356891451496],
            [-1.5966420919511339349, 0.0104786041905751727],
            [-1.7783427365938460074, 0.029151132638980264272]
        ]
        energy_previous = [
            [0, 1e-14], 
            [-0.0879115, 1.1293158298007674394e-06],
            [-2.25995, 0.0038263972837305035464],
            [-6.52141, 0.051998794872444117909],
            [-12.9855, 0.25049856933159531103],
            [-21.5192, 0.44650204517740099686],
        ]
        for macro in range(criteria.num_states()):
            self.assertAlmostEqual(
                lnpi_previous[macro][0],
                criteria.bias().ln_prob().value(macro),
                delta=5.*lnpi_previous[macro][1]
            )
            energy_analyzer = monte_carlo.analyze(monte_carlo.num_analyzers() - 1)
            energy_accumulator = energy_analyzer.analyze(macro).accumulator()
            stdev = (energy_previous[macro][1]**2 + energy_accumulator.block_stdev()**2)**(1./2.)
            self.assertAlmostEqual(
                energy_previous[macro][0],
                energy_accumulator.average(),
                #criteria.bias().ln_prob().value(macro),
                delta=10*stdev
            ) 


In [2]:
%%time
unittest.main(argv=[''], verbosity=2, exit=False)

test_serial_5max (__main__.TestFlatHistogramLJ)
Compare the free energies and potential energies with the NIST SRSW ... 

CPU times: user 1min 27s, sys: 1.01 s, total: 1min 28s
Wall time: 1min 28s


ok

----------------------------------------------------------------------
Ran 1 test in 88.649s

OK


Did this tutorial work as expected? Did you find any inconsistencies or have any comments? Please [contact](../../../CONTACT.rst) us. Any feedback is appreciated!